In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime
import lightgbm as lgb
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
import os
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score
import json
import ast
import time
from sklearn import linear_model
import eli5
from eli5.sklearn import PermutationImportance
import shap

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_selection import GenericUnivariateSelect, SelectPercentile, SelectKBest, f_classif, mutual_info_classif, RFE
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from catboost import CatBoostClassifier

## Loading Data

In [ ]:
train = pd.read_csv('../input/dont-overfit-ii/train.csv')
test = pd.read_csv('../input/dont-overfit-ii/test.csv')
labels = train.columns.drop(['id', 'target'])
train.head()

## EDA

In [ ]:
print("Train Shape: " , train.shape , "\nTest Shape:" , test.shape)

In [ ]:
train.info()

Check if there are any null values. Based upon the problem, there shouldn't be any.

In [ ]:
train.isnull().any().any()

Check the count of the target values

In [ ]:
train['target'].value_counts()

In [ ]:
train['target'].value_counts().plot(kind='bar', title='Count (target)')

Here, I'm going to plot the standard deviations on a histogram.
A histogram is different than a bar chart because a histogram works on continuous data and the bars are connected whereas a bar chart works on categorical data and gives info about the count of each category.
Here, we can see that the standard deviation of the columns follows a somewhat normal distribution, and that most of the standard deviations are between 0.9 and 1.10

In [ ]:
train[train.columns[2:]].std().plot('hist')
plt.title('Distribution of the Standard Deviations of the Features')

Here, I look at the distrubtion of the means of the features. We can see that the mean of the features is between -0.2 and +0.2, with a majority of the features centered around 0

In [ ]:
train[train.columns[2:]].mean().plot('hist')
plt.title('Distribution of the Means of the Features')

Here is a plot of the target variable that we will be predicting. We can see that with the test data, there are more 1's than 0's.

In [ ]:
train['target'].value_counts().plot('bar')

Here are the distributions of the first 28 columns. Here, we can see that the values in the columns are more or less similar. 

In [ ]:
print('Distributions of the first 28 columns')
plt.figure(figsize=(26,24))
for i, col in enumerate(list(train.columns)[2:30]):
    plt.subplot(7,4,i+1)
    plt.hist(train[col])
    plt.title(col)

## Basic Modeling

Creating the train and test sets

In [ ]:
X_train = train.drop(['id','target'], axis=1)
y_train = train['target']
X_test = test.drop(['id'], axis=1)

Scaling the data. Fit/Transform the scaler on the training data. Then fit it to the test data. Applying the fit or the fit_transform functions returns an ndarray. This needs to be converted to a DataFrame

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(ratio='minority', n_jobs=-1)
X_sm, y_sm = smote.fit_resample(X_train, y_train)
#outputs X_sm and y_sm as ndarrays, need to convert back to df
X_train = pd.DataFrame(X_sm, columns=labels)
y_train = pd.DataFrame(y_sm, columns=['target'])

In [ ]:
cols = X_train.columns
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = cols)
X = X_train
y = y_train

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns = cols)

Split the training data into training and validation data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# count of the y_train target variable now
y_t = pd.Series(y_train.iloc[:,0], name="training")
y_t.value_counts().plot('bar')

Using the random forest classifier, checking the cross validation score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
trees = 500
depth = 4
Model = RandomForestClassifier(n_estimators = trees, max_depth = depth, random_state = 0)
cv_scores = cross_val_score(Model, X_train, y_train, cv=5)
print("Cross validation score: %s" % cv_scores.mean())

Train the random forest model with the full training set now

In [ ]:
Model.fit(X, y)
y_test_pred = Model.predict(X_test)

In [ ]:
submission = pd.DataFrame({
    "id": test["id"],
    "target": y_test_pred
})
submission.to_csv('submission.csv', index=False)

Now, instead of using the random forest, we will use the Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GBModel = GradientBoostingClassifier()
GBModel.fit(X_train, y_train)
y_predict = GBModel.predict(X_val)
print('accuracy is', accuracy_score(y_predict, y_val))

In [ ]:
y_test_pred = GBModel.predict(X_test)

In [ ]:
GBsubmission = pd.DataFrame({
    "id": test["id"],
    "target": y_test_pred
})
GBsubmission.to_csv('GBSubmission.csv', index=False)